<a href="https://colab.research.google.com/github/markste-in/collatz/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [2]:
!lsblk

NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT
loop0         7:0    0   25M  1 loop /snap/amazon-ssm-agent/4046
loop2         7:2    0 55.5M  1 loop /snap/core18/2074
loop3         7:3    0 32.3M  1 loop /snap/snapd/12398
loop4         7:4    0 33.3M  1 loop /snap/amazon-ssm-agent/3552
loop5         7:5    0 32.3M  1 loop /snap/snapd/12704
loop6         7:6    0 55.4M  1 loop /snap/core18/2128
nvme1n1     259:0    0  200G  0 disk /data
nvme0n1     259:1    0  105G  0 disk 
└─nvme0n1p1 259:2    0  105G  0 part /


In [3]:
!sudo file -s /dev/nvme1n1

/dev/nvme1n1: SGI XFS filesystem data (blksz 4096, inosz 512, v2 dirs)


In [4]:
#!sudo mkfs -t xfs /dev/nvme1n1

In [6]:
!sudo mkdir /data
!sudo mount /dev/nvme1n1 /data
!sudo chown $USER:$USER /data

mkdir: cannot create directory ‘/data’: File exists
mount: /data: /dev/nvme1n1 already mounted on /data.


In [7]:
from multiprocessing import Pool
import psutil
from helper import *


In [8]:
print("CPU Count:", psutil.cpu_count())

CPU Count: 32


In [9]:
pool = Pool(psutil.cpu_count())

In [ ]:
df = create_all(pool)

## Going from  3 to 30000003 ##
Saved to /data/collatz_dbs/collatz_db_20210813-123327_133210042.parquet
Iteration took 933.001843214035
## Going from  30000003 to 60000003 ##


In [6]:
df.sample(20)


,Number,Length,Max,IsEven,nPrimes,Smallest_Prime,nSmallest_Prim,Biggest_Prime,nBiggest_Prime,isPrime
7735,97738,41,146608,True,2,2,1,48869,1,False
6643,96646,46,217456,True,4,2,1,191,1,False
4535,94538,85,227176,True,2,2,1,47269,1,False
3384,93387,85,420244,False,3,3,1,4447,1,False
1331,91334,134,205504,True,2,2,1,45667,1,False
6757,96760,98,275560,True,4,2,3,59,1,False
8457,98460,98,186928,True,4,2,2,547,1,False
4144,94147,160,815344,False,2,31,1,3037,1,False
1599,91602,209,3173092,True,4,2,1,727,1,False
9723,99726,116,336580,True,4,2,1,1511,1,False
